In [1]:
import numpy as np
import pandas as pd

import gensim
import pickle
import scipy
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [2]:
class BertToW2v(torch.nn.Module):
    def __init__(self, bert_model_name, lin_shape_in, lin_shape_out, emb_layer): # -, 768, 100, 6
        super(BertToW2v, self).__init__()
        self.emb_layer = emb_layer
        self.bert_model = BertModel.from_pretrained(bert_model_name)
        #self.bert_model.eval()
        self.linear_model = torch.nn.Linear(lin_shape_in, lin_shape_out, bias=True) # bias?
        torch.nn.init.uniform_(self.linear_model.weight, -0.1, 0.1)
        
    def forward(self, input_sentence): # ожидаем уже токенизированное предложение
        encoded_layers, _ = self.bert_model(input_sentence)
        bert_output = encoded_layers[self.emb_layer][0][1]
        linear_output = self.linear_model(bert_output).unsqueeze(0)
        return linear_output

In [3]:
ozhegov_emb = pd.read_csv('rus/ozhegov/ozhegov_emb.csv')[['word', 'definition']]
ozhegov_no_emb = pd.read_csv('rus/ozhegov/ozhegov_no_emb.csv')

dfreq = pd.read_csv('rusfreq/freqrnc2011.csv', sep = '\t')

model_name = 'v38ep_l6'

In [4]:
dfreq['percentile'] = pd.qcut(dfreq['Freq(ipm)'], 10, labels=False)

In [5]:
dfreq[dfreq['percentile'] >= 4].sort_values('Freq(ipm)')

,Lemma,PoS,Freq(ipm),R,D,Doc,percentile
49656,целить,v,1.4,54,85,94,4
6576,выковыривать,v,1.4,59,89,107,4
49822,цитирование,s,1.4,55,87,99,4
29432,пахота,s,1.4,59,87,89,4
13465,здравница,s,1.4,38,80,73,4
29402,патронаж,s,1.4,53,86,106,4
29400,патриций,s,1.4,50,75,68,4
37057,пугливо,adv,1.4,47,83,88,4
29367,патетика,s,1.4,57,89,99,4
13436,звукозапись,s,1.4,56,87,89,4


In [6]:
dfreq.sort_values('Freq(ipm)', ascending=False)

,Lemma,PoS,Freq(ipm),R,D,Doc,percentile
13824,и,conj,35801.8,100,99,37704,9
3710,в,pr,31374.2,100,98,37865,9
23328,не,part,18028.0,100,97,33999,9
22019,на,pr,15867.3,100,98,36748,9
51957,я,spro,12684.4,100,95,17116,9
3683,быть,v,12160.7,100,98,34184,9
27062,он,spro,11791.1,100,95,28132,9
40004,с,pr,11311.9,100,99,35700,9
50367,что,conj,8354.0,100,98,32419,9
0,а,conj,8198.0,100,97,32332,9


In [7]:
ozhegov_emb = pd.read_csv('rus/ozhegov/ozhegov_emb.csv')[['word', 'definition']]
ozhegov_no_emb = pd.read_csv('rus/ozhegov/ozhegov_no_emb.csv')

with open('rus/freq/low_freq.pkl', 'rb') as f:
    ulfreq = pickle.load(file=f)
    
ozhegov = pd.concat([ozhegov_emb, ozhegov_no_emb], axis = 0)

ozhegov['lfreq'] = ozhegov['word'].apply(lambda word: word in ulfreq)

ozhegov_lfreq = ozhegov[ozhegov['lfreq'] == True][['word', 'definition']]

In [8]:
defs = set(ozhegov_lfreq['word'])
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
w2v = gensim.models.KeyedVectors.load_word2vec_format('w2v_models/all_norm-sz500-w10-cb0-it3-min5.w2v', binary=True, unicode_errors='ignore')

test = pd.read_csv('russe-evaluation/russe/evaluation/test.csv')

In [9]:
bw2v = BertToW2v('bert-base-multilingual-cased', lin_shape_in=768, lin_shape_out=500, emb_layer=6) # !!!
bw2v.load_state_dict(torch.load(f'models/{model_name}.mdl')) # !!!
bw2v.to('cuda');

In [10]:
def find_embedding(word, model, w2v, ozhegov_lfreq, defs, tokenizer):
    if word in defs:
        print(word)
        defin = ozhegov_lfreq[ozhegov_lfreq['word'] == word].reset_index()['definition'][0]
        defin = '[CLS] [MASK] - ' + defin + ' [SEP]'
        tokens = tokenizer.tokenize(defin)
        tok_ids = torch.tensor([tokenizer.convert_tokens_to_ids(tokens)]).to('cuda')
        with torch.no_grad():
            embedding = model(tok_ids)
        return embedding.to('cpu').numpy()
    else:
        try:
            return w2v.get_vector(word)
        except KeyError:
            return np.nan
        
def get_cosine_distance(word1, word2, model, w2v, ozhegov_lfreq, defs, tokenizer):
    emb1 = find_embedding(word1, model, w2v, ozhegov_lfreq, defs, tokenizer)
    emb2 = find_embedding(word2, model, w2v, ozhegov_lfreq, defs, tokenizer)
    if (np.isnan(np.sum(emb1)) or np.isnan(np.sum(emb2))):
        return np.nan
    return 1 - scipy.spatial.distance.cosine(emb1, emb2)

#assert abs(get_cosine_distance('абрикос', 'год', bw2v, w2v, ozhegov_lfreq, defs, tokenizer) - (1 - w2v.distance('абрикос', 'год'))) < 0.000001

In [11]:
get_cosine_distance('диск-жокей', 'зачем-то', bw2v, w2v, ozhegov_lfreq, defs, tokenizer)

nan

In [12]:
# defin = ozhegov_lfreq[ozhegov_lfreq['word'] == 'зачем-то'].reset_index()['definition'][0]
# defin = '[CLS] [MASK] - ' + defin + ' [SEP]'
# tokens = tokenizer.tokenize(defin)
# tok_ids = torch.tensor([tokenizer.convert_tokens_to_ids(tokens)]).to('cuda')

In [13]:
# with torch.no_grad():
#     tmp = bw2v.bert_model(tok_ids)

In [14]:
test['sim'] = test.apply(lambda row: get_cosine_distance(row['word1'], row['word2'], bw2v, w2v, ozhegov_lfreq, defs, tokenizer), axis=1)

авангардизм
автовокзал
автовокзал
автовокзал
погрузчик
ажур
ажур
киноартист
аметист
аметист
анаконда
анаконда
анемия
малокровие
анемия
антисанитария
антисанитария
фармацевт
рефери
благоухание
радиоэлектроника
аэрация
аэрация
аэрация
аэрация
интоксикация
басок
грызня
бахча
баштан
бахча
бахча
бахча
бахча
бахча
баштан
бахча
баштан
баштан
баштан
безвкусица
бейсбол
лесопарк
лгун
ловкач
плут
прохиндей
шельма
бесчестье
бесчестье
биатлонист
жертвователь
бобина
бобина
бобина
бобина
нокаут
рефери
тугодум
чурбан
ломота
мученичество
надлом
пароксизм
прострел
резь
обувка
брюква
брюква
брюква
брюква
брюква
брюква
бука
бяка
бука
бутафория
бутафория
времяпровождение
маховик
вельвет
вельвет
вермишель
вермишель
умонастроение
амальгама
окрошка
набивка
гегемония
воздаяние
воздаяние
воздаяние
воздаяние
воздаяние
воздаяние
воздаяние
воздаяние
воздаяние
воздаяние
воздаяние
воздаяние
воздаяние
воздаяние
мщение
воздаяние
воздаяние
воздаяние
отмщение
воздаяние
воздаяние
воздаяние
воздаяние
воздаяние
воздаяние
в

телецентр
интервент
телецентр
телецентр
телецентр
телецентр
телецентр
телецентр
субъективизм
долготерпение
промокашка
пропись
куртизанка
супружница
ткачество
ткачество
ткачество
ткачество
тол
тол
тол
тол
тротил
дровосек
колун
провоз
миокард
нелады
разброд
экзарх
тритон
тритон
стежка
водосток
дымоход
тромбон
юниор
фланговый
гидроэлектростанция
выселок
тщание
тщание
тщание
тщание
аркада
барка
виноделие
кряж
физкультурник
стапель
упырь
упырь
упырь
вурдалак
упырь
упырь
упырь
поперечник
улан
комикс
шляхтич
времяпровождение
книгопечатание
контрибуция
самообразование
учительство
учительство
стачка
свиноферма
циркуляр
буй
ретро
флегматик
флегматик
флегматик
флегматик
краснознаменный
астрология
беднота
ветеринария
дека
мультипликатор
палаццо
радиотехника
руно
тайнопись
фураж
фураж
фураж
фураж
благотворитель
физкультурник
лесопарк
хорист
хорист
ваятель
флорист
дока
цирюльник
брадобрей
цирюльник
цирюльник
цирюльник
чары
чары
чары
чары
чары
чары
единорог
чары
чары
чары
чары
чары
чары
чары
чары
чар

In [15]:
test['sim'] = test['sim'].fillna(test['sim'].mean())

In [16]:
test.to_csv(f'russe-evaluation/russe/evaluation/{model_name}.csv', index=None)

In [17]:
def find_embedding_percentile(word, model, w2v, ozhegov_lfreq, defs, tokenizer, percentile, dfreq): # percentile: target (<=)
    if (dfreq[dfreq['Lemma'] == word].shape[0] != 1):
        try:
            return w2v.get_vector(word)
        except KeyError:
            return np.nan
    if (dfreq[dfreq['Lemma'] == word].iloc[0]['percentile'] > percentile):
        try:
            return w2v.get_vector(word)
        except KeyError:
            return np.nan
    return find_embedding(word, model, w2v, ozhegov_lfreq, defs, tokenizer)

def get_cosine_distance_percentile(word1, word2, model, w2v, ozhegov_lfreq, defs, tokenizer, percentile, dfreq):
    emb1 = find_embedding_percentile(word1, model, w2v, ozhegov_lfreq, defs, tokenizer, percentile, dfreq)
    emb2 = find_embedding_percentile(word2, model, w2v, ozhegov_lfreq, defs, tokenizer, percentile, dfreq)
    if (np.isnan(np.sum(emb1)) or np.isnan(np.sum(emb2))):
        return np.nan
    return 1 - scipy.spatial.distance.cosine(emb1, emb2)

In [18]:
percentiles = [0, 1, 2, 3]

for percentile in percentiles:
    sim = test.apply(lambda row: get_cosine_distance_percentile(row['word1'], row['word2'], bw2v, w2v, ozhegov_lfreq, defs, tokenizer, percentile, dfreq), axis=1)
    test['sim'] = sim
    test['sim'] = test['sim'].fillna(test['sim'].mean())
    test.to_csv(f'russe-evaluation/russe/evaluation/{model_name}_percentile{(percentile*10+10)}.csv')

авангардизм
анаконда
анаконда
малокровие
антисанитария
антисанитария
рефери
аэрация
аэрация
аэрация
аэрация
баштан
баштан
баштан
баштан
баштан
жертвователь
рефери
тугодум
прострел
брюква
брюква
брюква
брюква
брюква
брюква
бука
бука
времяпровождение
вельвет
вельвет
амальгама
рекреация
гидроузел
греча
депутация
депутация
депутация
депутация
уплотнитель
движитель
гипотенуза
директорат
присылка
единоначалие
единоначалие
супружница
засечка
снеговик
иван-чай
иван-чай
иван-чай
иван-чай
идолопоклонник
идолопоклонник
идолопоклонник
идолопоклонник
идолопоклонник
идолопоклонник
идолопоклонник
идолопоклонник
идолопоклонник
идолопоклонник
идолопоклонник
идолопоклонник
идолопоклонник
идолопоклонник
выжимка
бзик
гипотенуза
ряженка
блокадник
корж
двуличие
теплоцентраль
греча
засечка
мордва
хвороба
хвороба
леса
объективизм
иллюстратор
трясучка
океанология
краснобай
неолит
самоназвание
перепелка
блокадник
легат
бирюлька
мыза
патер
присылка
похудание
похудание
дипломант
полупустыня
позитрон
бирюлька
финт

валка
низменность
лютеранин
поисковик
поисковик
безделица
бесчестье
бирюлька
гравер
дзот
многоборье
мыза
систематика
помета
помета
помета
помета
помета
помета
патер
игуменья
протопоп
рей
латиница
присылка
рассыльный
похудание
похудание
правоведение
дипломант
пароксизм
атолл
полупустыня
протечка
халифат
позитрон
протопоп
протопоп
пруток
пруток
пруток
пруток
пук
грампластинка
директория
зачин
безделица
бирюлька
финтифлюшка
усач
забулдыга
пропойца
пьянчуга
бесчестье
правобережье
думка
осадка
палеолит
рассыльный
микрокосм
початок
цветоводство
чартер
реноме
реноме
реноме
реноме
репей
репей
полесье
рококо
рококо
гидроузел
пресвитер
засечка
рубчик
коллаборационист
саламандра
амфибия
саламандра
искуситель
сахароза
сахароза
сахароза
сахароза
хинди
оледенение
электрогитара
электросварщик
свиноферма
свиноферма
патер
пресвитер
церковник
геральдика
настроенность
популяризатор
реноме
отгадка
капельмейстер
статс-секретарь
бриг
квинтет
запарка
креатура
сталактит
сталактит
сталактит
сталагмит
сталактит

игуменья
иерей
протопоп
порицание
порицание
порицание
порицание
порицание
порицание
порицание
порицание
виолончелист
контрудар
рей
латиница
присылка
рассыльный
похудание
похудание
антрополог
петиция
правоведение
верховенство
главенство
жеребьевка
уголовщина
привоз
привоз
привоз
привоз
привоз
привоз
привоз
привоз
привоз
привоз
привоз
привоз
привоз
привоз
привоз
привоз
дипломант
медалист
пароксизм
атолл
оттяжка
портик
полупустыня
протечка
антагонист
конкурсант
супостат
халифат
позитрон
протопоп
протопоп
пруток
пруток
пруток
пруток
пук
лесопарк
грампластинка
директория
зачин
безделица
бирюлька
финтифлюшка
усач
забулдыга
пропойца
пьянчуга
бесчестье
магнитола
правобережье
радиотехника
мародерство
раздолье
раздолье
раздолье
раздолье
раздолье
раздолье
думка
осадка
ученичество
мидия
нелады
палеолит
разброд
рассыльный
растопка
растопка
растопка
растопка
растопка
растопка
антагонист
вегетация
микрокосм
початок
цветоводство
чартер
реноме
реноме
реноме
реноме
репей
репей
полесье
мазня
рокировка
ро

In [19]:
pd.set_option('max_colwidth', 160)

In [20]:
ozhegov_lfreq.shape

(8503, 2)

In [21]:
ozhegov_lfreq.sample(20)

,word,definition
22945,универсам,"универсальный магазин самообслуживания - магазин, торгующий продовольственными товарами, а также некоторыми товарами хозяйственного назначения"
845,национально-освободительный,направленный на борьбу за независимость своего народа нации
14468,ровнять,"делать ровным, гладким"
13076,сковырнуть,"ковыряя, сорвать, удалить"
8537,мочегонный,усиливающий выделение мочи
20235,сапа,глубокий окоп в сторону противника для постепенного приближения к нему при наступлении
36061,бредень,"небольшой невод, которым ловят рыбу вдвоем, идя бродом"
18678,антисанитарный,"противоречащий правилам санитарии, грязный и вредный для здоровья"
35508,объездчик,специалист по объездке лошадей
24506,трудоустроиться,"найти работу, стать трудоустроенным"


In [22]:
## необязательная часть, проверка:
with open('tmp_words.txt', 'r') as f:
    words = set(f.read().split('\n'))
    print(len(words))

2683


In [23]:
for word in words:
    if word != '':
        print(ozhegov_lfreq[ozhegov_lfreq['word'] == word].reset_index()['definition'][0])

свист, произведенный таким приспособлением


IndexError: index out of bounds